In [1]:
### RUN EVERY TIME: COMTRADE DATASETS EXTRACTING

In [2]:
### RUN EVERY TIME: INITIALIZATION

import pandas as pd
import numpy as np
import requests
import json
import gc
import os
import datetime
import time
import itertools

In [3]:
### RUN EVERY TIME: VERSION CONTROL

from platform import python_version
print('python version: ', python_version())
print('numpy version: ', np.__version__)
print('pandas version: ', pd.__version__)

python version:  3.7.4
numpy version:  1.17.2
pandas version:  0.25.3


In [4]:
### RUN EVERY TIME: MAIN CONSTANTS

### MultiIndex level slice constant:
All = slice(None)
### Universe path:
str_path_universe = 'Data_Files/Source_Files/acadian_universe.xlsx'
### Dates:
str_date_end = '2022-12-31'
date_start = pd.Timestamp('1989-12-29')
date_end = pd.Timestamp(str_date_end)
date_ison = pd.Timestamp('1994-12-31')
### NA for MS Excel files:
list_na_excel_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null',
                        '#N/A Requesting Data...', '#N/A Invalid Security', '#N/A Field Not Applicable', '---']
### Checked EBOPS service IDs list (df_serv_to_gics['GICS Group Code']):
list_services = ['206', '210', '214', '218', '219', '223', '227', '231', '232', '237', '240', '246', '247', '250', '251', '254', '255', '256', '257', '258', '263',
                 '264', '269', '272', '273', '288', '289', '292', '293', '294', '310', '391', '431', '500', '888', '891', '892', '894', '950']
### UN Comtrade authentication:
unc_login = 'pavelb'
unc_pass = 'bodoapux'
unc_token = 'wqgBfTCn0Idq0LZWWAFKgj3YQYRKczgdnfmlQ3CkanmvQzoAlnL1oK1OJ0yVoCjSLjkUozAj0/dD4eCkSLJO/6pLCqK+iweXMqMazaADI+YqBOUPFySpbXM0CEZepZEuNl5bqxg50EPVB5lCrifsoA=='
### UN Comtrade raw data containers:
str_path_unc_raw_comm_annual = 'Data_Files/Source_Files/unc_raw_comm_annual.h5'
str_path_unc_raw_serv_annual = 'Data_Files/Source_Files/unc_raw_serv_annual.h5'
str_key_unc_raw = 'unc_raw'
### UN Comtrade adopted data containers:
str_path_unc_res_all_annual = 'Data_Files/Source_Files/unc_res_all_annual.h5'
str_key_unc_res = 'unc_res'
### File with aggregated flows:
str_path_unc_res_flows = 'Data_Files/Source_Files/unc_res_flows.h5'
### Universal HDF5 key:
str_key_unc_res = 'unc_res'
### Augmented bilateral export:
str_path_export_bilateral = 'Data_Files/Source_Files/comtrade_export_bilateral.h5'
### Export key:
str_key_unc_export = 'export_augmented'
### Augmented bilateral import:
str_path_import_bilateral = 'Data_Files/Source_Files/comtrade_import_bilateral.h5'
### Import key:
str_key_unc_import = 'import_augmented'

In [5]:
### DEFINING COUNTRY CODES EXTRACTOR

def get_country_codes(use_local_copy = False):  
    ### In case if URL is unavailable:
    if (use_local_copy):
        url_country_code = 'Data_Files/Source_Files/countrycode.html'
    ### Online extraction:
    else:
        url_country_code = 'https://countrycode.org/'
    df_full_codes = pd.read_html(url_country_code, index_col = 'COUNTRY')[0]
    df_full_codes[['ISO SHORT', 'ISO LONG']] = df_full_codes['ISO CODES'].str.split(' / ', expand = True)
    df_result = df_full_codes[['ISO SHORT', 'ISO LONG']].sort_index()    
    df_result.index = df_result.index.str.upper()
    ### Results output:
    return df_result

In [6]:
### DEFINING EXTRACTION UNIVERSE DATA FROM MS EXCEL SOURCE (TO BE IGNORED IN PRODUCT CODE)

def ison_membership_converting(str_path_universe, date_end, bool_daily = False, int_backfill_months = 0):
    ### Defining business-month-end reindexation on country level:
    def country_modify(ser_raw_country, date_end):
        ser_res_country = ser_raw_country.droplevel(0).resample('MS').last().resample('BM').last()
        range_country = pd.date_range(ser_res_country.index[0], date_end, freq = 'BM')
        return ser_res_country.reindex(range_country).ffill()
    ### Markets encoding table:
    dict_markets = {50 : 'DM', 57 : 'EM', 504 : 'FM', 0: np.NaN}     
    ### Loading source file:
    df_raw_universe = pd.read_excel(engine = 'openpyxl', io = str_path_universe, sheet_name = 'Switchers', header = 0, parse_dates = True, index_col = [0, 1],
                                 na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                             '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null'], keep_default_na = False)
    ### Converting source file:
    df_raw_universe.index.names = ['Country', 'Date']
    ser_raw_universe = df_raw_universe['Region']
    ser_raw_universe.fillna(0, inplace = True)
    ser_raw_universe.name = 'Market'
    ### By country reindexation and translation:
    ser_res_universe = ser_raw_universe.groupby('Country').apply(country_modify, date_end)
    ser_res_universe.index.names = ['Country', 'Date']
    ser_res_universe = ser_res_universe.replace(dict_markets).reorder_levels([1, 0]).sort_index() 
    ### Expanding membership for primary regions members by backfilling:
    if int_backfill_months:
        ### List of regions:
        list_region = list(ser_res_universe.dropna().unique())
        ### Initialising of collection of series with backfilled data for each region:
        list_ison_backfill = []
        ### Regions looping:
        for iter_region in list_region:
            ### Defining start of region date:
            date_first_valid = ser_res_universe.loc[ser_res_universe == iter_region].first_valid_index()[0]
            ### Creating dates index to backfilling:
            idx_date_backfill = pd.date_range(end = date_first_valid, periods = int_backfill_months + 1, freq = 'BM')[: -1]
            ### Creating primary countries index to backfilling:            
            idx_region_backfill = ser_res_universe.loc[ser_res_universe == iter_region].loc[date_first_valid, All].index.get_level_values('Country')
            ### Creating full index:
            idx_ison_backfill = pd.MultiIndex.from_product([idx_date_backfill, idx_region_backfill])
            ### Series with backfilled data:
            list_ison_backfill.append(pd.Series(iter_region, index = idx_ison_backfill))
        ### Combination of backfilled series and original ISON data:    
        ser_res_universe = ser_res_universe.combine_first(pd.concat(list_ison_backfill, axis = 0)).sort_index()  
        ser_res_universe.index.names = ['Date', 'Country']
    ### Converting to daily frequency:
    if bool_daily:
        ser_res_universe = ser_res_universe.reset_index('Country').groupby('Country').resample('B').ffill()['Market'].swaplevel().sort_index()    
    ### Results output:
    ser_res_universe.name = 'Market'
    return ser_res_universe

In [7]:
### RUN EVERY TIME: COMMON DATA EXTRACTION STEPS

### World Country Codes:
df_country_codes = get_country_codes()
### ISON membership history:
ser_ison_membership = ison_membership_converting(str_path_universe, pd.to_datetime(str_date_end))
ser_ison_membership.index.names = ['Date', 'Reporter']
### ISON Members:
list_ison_countries = sorted(ser_ison_membership.index.get_level_values('Reporter').unique())
### ISON status for the last available date:
ser_ison_status = ser_ison_membership.loc[ser_ison_membership.index[-1][0]]

In [8]:
### UN COMTRADE: DATA EXTRACTION

In [ ]:
### UN COMTRADE: CODELISTS LOADING (EXPANDED VERSION)

### Codelists container:
dict_codelist = {}
### List of reference tables for categories:
request_session = requests.Session()
obj_unc_reference = request_session.get('https://comtradeapi.un.org/files/v1/app/reference/ListofReferences.json')
df_cat_reference = pd.DataFrame(obj_unc_reference.json()['results']).set_index('category')
### Description of categories
df_cat_description = pd.DataFrame(request_session.get(df_cat_reference.loc['dataitem']['fileuri']).json()['results']).set_index('dataItem')
### Parameters:
print('typeCode:')
display(df_cat_description.loc['typeCode'])
print('---')
print('Options:')
ser_type_code = pd.Series(['Commodity', 'Service'], index = ['C', 'S'])
ser_type_code.index.names = ['id']
ser_type_code.name = 'typeCode'
display(ser_type_code)
dict_codelist['typeCode'] = ser_type_code
print('---')
print('---')
print('freqCode:')
display(df_cat_description.loc['freqCode'])
print('---')
print('Options:')
ser_freq_code = pd.DataFrame(request_session.get(df_cat_reference.loc['freq']['fileuri']).json()['results']).set_index('id').squeeze()
display(ser_freq_code)
dict_codelist['freqCode'] = ser_freq_code
print('---')
print('---')
print('clCode (Commodities):')
display(df_cat_description.loc['classificationCode'])
print('---')
print('Options:')
df_hs_comm = pd.DataFrame(request_session.get(df_cat_reference.loc['cmd:HS']['fileuri']).json()['results'])
ser_hs_comm_ag2 = df_hs_comm[df_hs_comm['aggrLevel'] == 2].drop(['parent', 'isLeaf', 'aggrLevel'], axis = 1).set_index('id').squeeze().str[5: ]
ser_hs_comm_ag2.name = 'clCode'
display(ser_hs_comm_ag2)
dict_codelist['clCode'] = {}
dict_codelist['clCode']['C'] = ser_hs_comm_ag2
print('---')
print('---')
print('clCode (Services):')
display(df_cat_description.loc['classificationCode'])
print('---')
print('Options:')
df_eb_serv = pd.DataFrame(request_session.get(df_cat_reference.loc['cmd:EB']['fileuri']).json()['results'])
df_eb_serv_ag2 = df_eb_serv[df_eb_serv['parent'].isin(df_eb_serv[df_eb_serv['parent'] == '200']['id'])]
ser_eb_serv_ag2 = df_eb_serv_ag2[df_eb_serv_ag2['id'].astype(int) <= 950].set_index('id')['text']
ser_eb_serv_ag2.name = 'clCode'
display(ser_eb_serv_ag2)
dict_codelist['clCode']['S'] = ser_eb_serv_ag2
print('---')
print('---')
print('reporterCode:')
display(df_cat_description.loc['reporterCode'])
print('---')
df_reporter_raw = pd.DataFrame(request_session.get(df_cat_reference.loc['reporter']['fileuri']).json()['results'])
df_reporter_raw['id'] = df_reporter_raw['id'].astype(str).str.zfill(3)
df_reporter_raw['entryEffectiveDate'] = pd.to_datetime(df_reporter_raw['entryEffectiveDate'])
df_reporter_raw['entryExpiredDate'] = pd.to_datetime(df_reporter_raw['entryExpiredDate'])
df_reporter_raw = df_reporter_raw[df_reporter_raw['reporterCodeIsoAlpha2'].isin(df_country_codes['ISO SHORT'])]
df_reporter_raw = df_reporter_raw[df_reporter_raw['entryExpiredDate'].isna() | (df_reporter_raw['entryExpiredDate'] > date_start)]
ser_reporter_code = df_reporter_raw.set_index('id')['reporterCodeIsoAlpha2'].squeeze()
ser_reporter_code.name = 'Reporter'
display(ser_reporter_code)
dict_codelist['reporterCode'] = ser_reporter_code
print('---')
print('---')
print('period:')
display(df_cat_description.loc['period'])
print('---')
print('---')
print('partnerCode:')
display(df_cat_description.loc['partnerCode'])
print('---')
df_partner_raw = pd.DataFrame(request_session.get(df_cat_reference.loc['partner']['fileuri']).json()['results'])
df_partner_raw['id'] = df_partner_raw['id'].astype(str).str.zfill(3)
df_partner_raw['entryEffectiveDate'] = pd.to_datetime(df_partner_raw['entryEffectiveDate'])
df_partner_raw['entryExpiredDate'] = pd.to_datetime(df_partner_raw['entryExpiredDate'])
df_partner_raw = df_partner_raw[df_partner_raw['PartnerCodeIsoAlpha2'].isin(df_country_codes['ISO SHORT'])]
df_partner_raw = df_partner_raw[df_partner_raw['entryExpiredDate'].isna() | (df_partner_raw['entryExpiredDate'] > date_start)]
ser_partner_code = df_partner_raw.set_index('id')['PartnerCodeIsoAlpha2'].squeeze()
ser_partner_code.name = 'Partner'
display(ser_partner_code)
dict_codelist['partnerCode'] = ser_partner_code
print('Non-reporting partners:')
print(sorted(set(ser_partner_code) - set(ser_reporter_code)))
print('---')
print('---')
print('partner2Code:')
display(df_cat_description.loc['partner2Code'])
print('---')
print('---')
print('cmdCode:')
display(df_cat_description.loc['cmdCode'])
print('---')
print('---')
print('flowCode:')
display(df_cat_description.loc['flowCode'])
print('---')
ser_flow_code = pd.DataFrame(request_session.get(df_cat_reference.loc['flow']['fileuri']).json()['results']).set_index('id').squeeze()
ser_flow_code.name = 'flowCode'
display(ser_flow_code)
dict_codelist['flowCode'] = ser_flow_code
print('---')
print('---')
print('customsCode:')
display(df_cat_description.loc['customsCode'])
print('---')
ser_customs_code = pd.DataFrame(request_session.get(df_cat_reference.loc['customs']['fileuri']).json()['results']).set_index('id').squeeze()
ser_customs_code.name = 'customsCode'
display(ser_customs_code)
dict_codelist['flowCode'] = ser_customs_code
print('---')
print('---')
print('motCode:')
display(df_cat_description.loc['motCode'])
print('---')
ser_tansport_code = pd.DataFrame(request_session.get(df_cat_reference.loc['mot']['fileuri']).json()['results']).set_index('id').squeeze()
ser_tansport_code.name = 'motCode'
display(ser_tansport_code)
dict_codelist['motCode'] = ser_tansport_code
print('---')
print('---')

In [9]:
### UN COMTRADE: CODELISTS LOADING (PRODUCT VERSION)

### Codelists container:
dict_codelist = {}
### List of reference tables for categories:
request_session = requests.Session()
obj_unc_reference = request_session.get('https://comtradeapi.un.org/files/v1/app/reference/ListofReferences.json')
df_cat_reference = pd.DataFrame(obj_unc_reference.json()['results']).set_index('category')
### Description of categories
df_cat_description = pd.DataFrame(request_session.get(df_cat_reference.loc['dataitem']['fileuri']).json()['results']).set_index('dataItem')
### Parameters:
ser_type_code = pd.Series(['Goods', 'Services'], index = ['C', 'S'])
ser_type_code.index.names = ['id']
ser_type_code.name = 'typeCode'
dict_codelist['typeCode'] = ser_type_code
### Frequency:
ser_freq_code = pd.DataFrame(request_session.get(df_cat_reference.loc['freq']['fileuri']).json()['results']).set_index('id').squeeze()
dict_codelist['freqCode'] = ser_freq_code
### Commodity HS Codes:
df_hs_comm = pd.DataFrame(request_session.get(df_cat_reference.loc['cmd:HS']['fileuri']).json()['results'])
ser_hs_comm_ag2 = df_hs_comm[df_hs_comm['aggrLevel'] == 2].drop(['parent', 'isLeaf', 'aggrLevel'], axis = 1).set_index('id').squeeze().str[5: ]
ser_hs_comm_ag2.name = 'clCode'
dict_codelist['clCode'] = {}
dict_codelist['clCode']['C'] = ser_hs_comm_ag2
### Service EBOPS Codes:
df_eb_serv = pd.DataFrame(request_session.get(df_cat_reference.loc['cmd:EB']['fileuri']).json()['results'])
df_eb_serv_ag2 = df_eb_serv[df_eb_serv['parent'].isin(df_eb_serv[df_eb_serv['parent'] == '200']['id'])]
ser_eb_serv_ag2 = df_eb_serv_ag2[df_eb_serv_ag2['id'].astype(int) <= 950].set_index('id')['text']
ser_eb_serv_ag2.name = 'clCode'
dict_codelist['clCode']['S'] = ser_eb_serv_ag2
### United codes:
dict_codelist['clCode']['T'] = pd.concat([dict_codelist['clCode']['C'], dict_codelist['clCode']['S']], axis = 0)
### Reporter Codes:
df_reporter_raw = pd.DataFrame(request_session.get(df_cat_reference.loc['reporter']['fileuri']).json()['results'])
df_reporter_raw['id'] = df_reporter_raw['id'].astype(str).str.zfill(3)
df_reporter_raw['entryEffectiveDate'] = pd.to_datetime(df_reporter_raw['entryEffectiveDate'])
df_reporter_raw['entryExpiredDate'] = pd.to_datetime(df_reporter_raw['entryExpiredDate'])
df_reporter_raw = df_reporter_raw[df_reporter_raw['reporterCodeIsoAlpha2'].isin(df_country_codes['ISO SHORT'])]
df_reporter_raw = df_reporter_raw[df_reporter_raw['entryExpiredDate'].isna() | (df_reporter_raw['entryExpiredDate'] > date_start)]
ser_reporter_code = df_reporter_raw.set_index('id')['reporterCodeIsoAlpha2'].squeeze()
ser_reporter_code.name = 'Reporter'
dict_codelist['reporterCode'] = ser_reporter_code
### Partner Codes:
df_partner_raw = pd.DataFrame(request_session.get(df_cat_reference.loc['partner']['fileuri']).json()['results'])
df_partner_raw['id'] = df_partner_raw['id'].astype(str).str.zfill(3)
df_partner_raw['entryEffectiveDate'] = pd.to_datetime(df_partner_raw['entryEffectiveDate'])
df_partner_raw['entryExpiredDate'] = pd.to_datetime(df_partner_raw['entryExpiredDate'])
df_partner_raw = df_partner_raw[df_partner_raw['PartnerCodeIsoAlpha2'].isin(df_country_codes['ISO SHORT'])]
df_partner_raw = df_partner_raw[df_partner_raw['entryExpiredDate'].isna() | (df_partner_raw['entryExpiredDate'] > date_start)]
ser_partner_code = df_partner_raw.set_index('id')['PartnerCodeIsoAlpha2'].squeeze()
ser_partner_code.name = 'Partner'
dict_codelist['partnerCode'] = ser_partner_code
### Trade Flow Codes:
ser_flow_code = pd.DataFrame(request_session.get(df_cat_reference.loc['flow']['fileuri']).json()['results']).set_index('id').squeeze()
ser_flow_code.name = 'flowCode'
dict_codelist['flowCode'] = ser_flow_code
### Customs Codes:
ser_customs_code = pd.DataFrame(request_session.get(df_cat_reference.loc['customs']['fileuri']).json()['results']).set_index('id').squeeze()
ser_customs_code.name = 'customsCode'
dict_codelist['customsCode'] = ser_customs_code
### Transport Codes:
ser_tansport_code = pd.DataFrame(request_session.get(df_cat_reference.loc['mot']['fileuri']).json()['results']).set_index('id').squeeze()
ser_tansport_code.name = 'motCode'
dict_codelist['motCode'] = ser_tansport_code

In [13]:
### UN COMTRADE: DATA REQUEST PARAMETERS

str_primary_key = 'e690550ab9414234a6b705220596677a'
#str_primary_key = '3f0b8d53d71b401e840d58c90a283176'
str_goods_type = 'C'
str_services_type = 'S'
str_freq = 'A'
str_goods_class = 'HS'
str_services_class = 'EB'
str_export_flow = 'X'
str_import_flow = 'M'
list_un_reporters = dict_codelist['reporterCode'].index.to_list()
list_un_partners = dict_codelist['partnerCode'].index.to_list()
list_un_goods_ag2 = dict_codelist['clCode'][str_goods_type].index.to_list()
list_un_services_ag2 = dict_codelist['clCode'][str_services_type].index.to_list()
list_periods = str_period = list(map(str, range(date_start.year, date_end.year + 1)))
int_pause_short_sec = 10
int_pause_long_sec = 10
int_period_portion = 5

In [14]:
### UN COMTRADE: DATA REQUEST EXECUTION

def get_un_comtrade_data(str_type, str_freq, str_classification, str_trade_flow, list_reporters, list_partners, list_commodities, list_periods, str_api_key, 
                         int_limit = 99999):
    ### Request preparation:
    str_url_base = 'https://comtradeapi.un.org/data/v1/get/'
    str_url_request = str_url_base + str_type + '/'
    str_url_request = str_url_request + str_freq + '/' 
    str_url_request = str_url_request + str_classification + '?'   
    str_url_request = str_url_request + 'flowCode=' + str_trade_flow 
    str_url_request = str_url_request + '&reporterCode=' + ','.join(list_reporters)
    str_url_request = str_url_request + '&partnerCode=' + ','.join(list_partners)    
    str_url_request = str_url_request + '&cmdCode=' + ','.join(list_commodities)
    str_url_request = str_url_request + '&customsCode=' + 'C00'
    str_url_request = str_url_request + '&motCode=' + '0'    
    str_url_request = str_url_request + '&partner2Code=' + '000'       
    str_url_request = str_url_request + '&period=' + ','.join(list_periods)  
    str_url_request = str_url_request + '&maxrecords=' + str(int_limit)
    ### Request sending:
    bool_loaded = False
    while (not bool_loaded):
        request_session = requests.Session()
        dict_request_headers = {}
        dict_request_headers['Cache-Control'] = 'no-cache'
        dict_request_headers['Ocp-Apim-Subscription-Key'] = str_api_key
        request_session.headers.update(dict_request_headers)
        ### Respond processing:
        print(str_url_request)    
        obj_unc_dataset = request_session.get(str_url_request)
        int_dataset_length = obj_unc_dataset.json()['count']
        if (int_dataset_length > 0):
            df_dataset_raw = pd.DataFrame(obj_unc_dataset.json()['data'])
            df_dataset_res = df_dataset_raw[['flowCode', 'typeCode', 'period', 'reporterCode', 'partnerCode', 'cmdCode', 'primaryValue']]
            df_dataset_res.loc[:, 'Flow'] = df_dataset_res['flowCode'].replace(dict_codelist['flowCode']).astype('category').values
            df_dataset_res['Flow'].cat.set_categories(sorted(dict_codelist['flowCode'].values), ordered = True, inplace = True)        
            df_dataset_res.loc[:, 'Type'] = df_dataset_res['typeCode'].replace(dict_codelist['typeCode']).astype('category').values
            df_dataset_res['Type'].cat.set_categories(sorted(dict_codelist['typeCode'].values), ordered = True, inplace = True)
            df_dataset_res.loc[:, 'Date'] = (pd.to_datetime(df_dataset_raw['period']) + pd.offsets.BYearEnd()).values
            df_dataset_res.loc[:, 'Reporter'] = df_dataset_res['reporterCode'].astype(str).str.zfill(3).replace(dict_codelist['reporterCode'])\
                                                                                                       .astype('category').values
            df_dataset_res['Reporter'].cat.set_categories(sorted(dict_codelist['partnerCode'].unique()), ordered = True, inplace = True)
            df_dataset_res.loc[:, 'Partner'] = df_dataset_res['partnerCode'].astype(str).str.zfill(3).replace(dict_codelist['partnerCode']).astype('category').values
            df_dataset_res['Partner'].cat.set_categories(sorted(dict_codelist['partnerCode'].unique()), ordered = True, inplace = True)
            df_dataset_res.loc[:, 'Commodity_ID'] = df_dataset_res['cmdCode'].astype('category').values
            df_dataset_res['Commodity_ID'].cat.set_categories(sorted(dict_codelist['clCode']['T'].index), ordered = True, inplace = True)
            df_dataset_res.loc[:, 'Value'] = (df_dataset_res['primaryValue'] / 1000).astype('int32').values
            ### Data clearing:
            df_dataset_res.drop(df_dataset_res[(df_dataset_res['Reporter'] == 'SA') & (df_dataset_res['Partner'] == 'TW')].index, inplace = True)
            df_dataset_res.drop(df_dataset_res[df_dataset_res['Reporter'] == df_dataset_res['Partner']].index, inplace = True)
            df_dataset_res['Value'].clip(lower = 0, inplace = True)
            df_dataset_res = df_dataset_res[['Date', 'Reporter', 'Partner', 'Flow', 'Type', 'Commodity_ID', 'Value']]#.dropna()
            print('Loaded Observations Number:', int_dataset_length)
            bool_loaded = True
        elif (int_dataset_length == 0):
            print('Empty Dataset')
            bool_loaded = True            
            df_dataset_res = None            
        else:
            print('Loading Error. Let\'s try once more...')
    return df_dataset_res

In [ ]:
### TEMP

str_type = 'C'
str_freq = 'A'
str_classification = 'HS'
str_trade_flow = 'M'
list_reporters = sorted(ser_reporter_code[ser_reporter_code == 'DE'].index)
list_partners = sorted(ser_partner_code[ser_partner_code == 'AE'].index)
list_commodities = ['95', '96']
list_periods = ['2008']
str_api_key = str_primary_key
int_limit = 99999

if True:
    bool_loaded = False
    ### Request preparation:
    str_url_base = 'https://comtradeapi.un.org/data/v1/get/'
    str_url_request = str_url_base + str_type + '/'
    str_url_request = str_url_request + str_freq + '/' 
    str_url_request = str_url_request + str_classification + '?'   
    str_url_request = str_url_request + 'flowCode=' + str_trade_flow 
    str_url_request = str_url_request + '&reporterCode=' + ','.join(list_reporters)
    str_url_request = str_url_request + '&partnerCode=' + ','.join(list_partners)    
    str_url_request = str_url_request + '&cmdCode=' + ','.join(list_commodities)
    str_url_request = str_url_request + '&customsCode=' + 'C00'
    str_url_request = str_url_request + '&motCode=' + '0'  
    str_url_request = str_url_request + '&partner2Code=' + '000'    
    str_url_request = str_url_request + '&period=' + ','.join(list_periods)  
    str_url_request = str_url_request + '&maxrecords=' + str(int_limit)
    while (not bool_loaded):
        request_session = requests.Session()
        dict_request_headers = {}
        dict_request_headers['Cache-Control'] = 'no-cache'
        dict_request_headers['Ocp-Apim-Subscription-Key'] = str_api_key
        request_session.headers.update(dict_request_headers)
        ### Respond processing:
        print(str_url_request)    
        obj_unc_dataset = request_session.get(str_url_request)
        int_dataset_length = obj_unc_dataset.json()['count']
        if (int_dataset_length > 0):
            df_dataset_raw = pd.DataFrame(obj_unc_dataset.json()['data'])
            df_dataset_res = df_dataset_raw[['flowCode', 'typeCode', 'period', 'reporterCode', 'partnerCode', 'cmdCode', 'primaryValue']]
            df_dataset_res.loc[:, 'Flow'] = df_dataset_res['flowCode'].replace(dict_codelist['flowCode']).astype('category').values
            df_dataset_res['Flow'].cat.set_categories(sorted(dict_codelist['flowCode'].values), ordered = True, inplace = True)        
            df_dataset_res.loc[:, 'Type'] = df_dataset_res['typeCode'].replace(dict_codelist['typeCode']).astype('category').values
            df_dataset_res['Type'].cat.set_categories(sorted(dict_codelist['typeCode'].values), ordered = True, inplace = True)
            df_dataset_res.loc[:, 'Date'] = (pd.to_datetime(df_dataset_raw['period']) + pd.offsets.BYearEnd()).values
            df_dataset_res.loc[:, 'Reporter'] = df_dataset_res['reporterCode'].astype(str).str.zfill(3).replace(dict_codelist['reporterCode']).astype('category').values
            df_dataset_res['Reporter'].cat.set_categories(sorted(dict_codelist['partnerCode'].unique()), ordered = True, inplace = True)
            df_dataset_res.loc[:, 'Partner'] = df_dataset_res['partnerCode'].astype(str).str.zfill(3).replace(dict_codelist['partnerCode']).astype('category').values
            df_dataset_res['Partner'].cat.set_categories(sorted(dict_codelist['partnerCode'].unique()), ordered = True, inplace = True)
            df_dataset_res.loc[:, 'Commodity_ID'] = df_dataset_res['cmdCode'].astype('category').values
            df_dataset_res['Commodity_ID'].cat.set_categories(sorted(dict_codelist['clCode']['T'].index), ordered = True, inplace = True)
            df_dataset_res.loc[:, 'Value'] = (df_dataset_res['primaryValue'] / 1000).astype('int32').values
            ### Data clearing:
            df_dataset_res.drop(df_dataset_res[(df_dataset_res['Reporter'] == 'SA') & (df_dataset_res['Partner'] == 'TW')].index, inplace = True)
            df_dataset_res.drop(df_dataset_res[df_dataset_res['Reporter'] == df_dataset_res['Partner']].index, inplace = True)
            df_dataset_res['Value'].clip(lower = 0, inplace = True)
            df_dataset_res = df_dataset_res[['Date', 'Reporter', 'Partner', 'Flow', 'Type', 'Commodity_ID', 'Value']]#.dropna()
            print(int_dataset_length, 'Observations loaded')
            bool_loaded = True
        elif (int_dataset_length == 0):
            print('Empty Dataset')
            bool_loaded = True            
        else:
            print('Loading Error. Let\'s try once more...')
#    if (int_dataset_length > 0):
#        return df_dataset_res
#    else:
#        return None

In [15]:
### DATA LOADING ENGINE

list_un_collection = []

#for iter_portion in range(-(-len(list_periods) // int_period_portion)):
for iter_portion in range(-(-len(list_periods) // int_period_portion))[3 : 4]:
    gc.collect()
    list_iter_periods = list_periods[iter_portion *  int_period_portion : (iter_portion + 1) *  int_period_portion]
#    print(iter_portion, list_iter_periods)
    ### Services data loading:
#    for iter_comm_id in list_un_services_ag2:      
    for iter_comm_id in list_un_services_ag2[ : 5]:  
        print(iter_comm_id, '/', list_iter_periods)        
        ### Export of Services:        
        df_iter_dataset = get_un_comtrade_data(str_services_type, str_freq, str_services_class, str_export_flow, list_un_reporters, list_un_partners, 
                                               [iter_comm_id], list_iter_periods, str_primary_key)
        if (df_iter_dataset is not None):
            list_un_collection.append(df_iter_dataset)
            time.sleep(int_pause_short_sec)
        else:
            time.sleep(int_pause_long_sec)                    
        ### Import of Services:        
        df_iter_dataset = get_un_comtrade_data(str_services_type, str_freq, str_services_class, str_import_flow, list_un_reporters, list_un_partners, 
                                               [iter_comm_id], list_iter_periods, str_primary_key)
        if (df_iter_dataset is not None):
            list_un_collection.append(df_iter_dataset)
            time.sleep(int_pause_short_sec)
        else:
            time.sleep(int_pause_long_sec)            
#        break
    ### Goods data loading:
#    for iter_comm_id in list_un_goods_ag2:
    for iter_comm_id in list_un_goods_ag2[-5 : ]:        
        print(iter_comm_id, '/', list_iter_periods)        
        ### Export of Goods:
        df_iter_dataset = get_un_comtrade_data(str_goods_type, str_freq, str_goods_class, str_export_flow, list_un_reporters, list_un_partners, 
                                               [iter_comm_id], list_iter_periods, str_primary_key)
        if (df_iter_dataset is not None):
            list_un_collection.append(df_iter_dataset)
            time.sleep(int_pause_short_sec)
        else:
            time.sleep(int_pause_long_sec) 
        ### Import of Goods:
        df_iter_dataset = get_un_comtrade_data(str_goods_type, str_freq, str_goods_class, str_import_flow, list_un_reporters, list_un_partners, 
                                               [iter_comm_id], list_iter_periods, str_primary_key)
        if (df_iter_dataset is not None):
            list_un_collection.append(df_iter_dataset)
            time.sleep(int_pause_short_sec)
        else:
            time.sleep(int_pause_long_sec)                    
#        break
#    break

206 / ['2004', '2005', '2006', '2007', '2008']
https://comtradeapi.un.org/data/v1/get/S/A/EB?flowCode=X&reporterCode=004,008,012,020,024,660,028,886,032,051,533,036,040,031,044,048,050,052,112,056,058,084,204,060,064,068,070,072,092,076,096,100,854,108,132,116,120,124,136,140,148,152,156,344,446,170,174,178,184,188,384,191,192,531,196,203,408,180,208,262,212,214,218,818,222,226,232,233,748,231,230,234,280,242,246,251,258,583,266,270,268,276,288,292,300,304,308,320,324,624,328,332,336,340,348,352,699,360,364,368,372,376,380,388,392,400,398,404,296,414,417,418,428,422,426,430,434,440,442,450,454,458,462,466,470,584,478,480,175,484,496,499,500,504,508,104,580,516,520,524,528,530,540,554,558,562,566,570,807,579,512,586,585,591,598,600,604,608,616,620,634,410,498,638,642,643,646,652,654,659,662,534,666,670,882,674,678,682,686,688,690,694,702,703,705,090,706,710,728,724,144,275,729,736,740,752,757,760,762,764,626,768,772,776,780,788,792,795,796,798,800,804,784,826,834,858,842,860,548,862,704

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pyda

Loaded Observations Number: 6322
https://comtradeapi.un.org/data/v1/get/S/A/EB?flowCode=M&reporterCode=004,008,012,020,024,660,028,886,032,051,533,036,040,031,044,048,050,052,112,056,058,084,204,060,064,068,070,072,092,076,096,100,854,108,132,116,120,124,136,140,148,152,156,344,446,170,174,178,184,188,384,191,192,531,196,203,408,180,208,262,212,214,218,818,222,226,232,233,748,231,230,234,280,242,246,251,258,583,266,270,268,276,288,292,300,304,308,320,324,624,328,332,336,340,348,352,699,360,364,368,372,376,380,388,392,400,398,404,296,414,417,418,428,422,426,430,434,440,442,450,454,458,462,466,470,584,478,480,175,484,496,499,500,504,508,104,580,516,520,524,528,530,540,554,558,562,566,570,807,579,512,586,585,591,598,600,604,608,616,620,634,410,498,638,642,643,646,652,654,659,662,534,666,670,882,674,678,682,686,688,690,694,702,703,705,090,706,710,728,724,144,275,729,736,740,752,757,760,762,764,626,768,772,776,780,788,792,795,796,798,800,804,784,826,834,858,842,860,548,862,704,876,887,894,7

In [28]:
### TEMP

list_periods

['2008']

In [16]:
### DATA CONCATENATION AND SAVING

### Downloaded Data Concatenating:
df_full_dataset = pd.concat(list_un_collection, axis = 0, sort = False, ignore_index = True)
del list_un_collection
ser_full_dataset = df_full_dataset.set_index(['Date', 'Reporter', 'Partner', 'Flow', 'Type', 'Commodity_ID']).squeeze().sort_index()
### Dataset saving:
ser_full_dataset.to_hdf(path_or_buf = str_path_unc_res_all_annual, key = str_key_unc_res, mode = 'w', format = 'table', complevel = 9)

In [ ]:
### EXPORT AND REVERTED IMPORT CONCATENATION

gc.collect()
### File deleting:
if (os.path.exists(str_path_unc_res_flows)):
    os.remove(str_path_unc_res_flows)
### Results container:
list_export_aug = []
### Countries portion length:
int_portion = 5
list_unc_countries = sorted(dict_codelist['partnerCode'].unique())
### Looping over countrie portions:
for iter_num in range(len(list_unc_countries) // int_portion + 1):
#for iter_num in range(2):
    gc.collect()    
    ### Portion of countries selecting:
    list_iter_countries = list(list_unc_countries)[int_portion * iter_num : int_portion * (iter_num + 1)]
    if (len(list_iter_countries) > 0):
        print(list_iter_countries)
        ### Export data loading:
        ser_unc_export = pd.read_hdf(str_path_unc_res_all_annual, key = str_key_unc_res,
                                     where = "(Flow = 'Export') & (Reporter in list_iter_countries) & (Partner != 'World')").droplevel('Flow')
        print('Export dataset loaded')
        ### Import data loading:
        ser_unc_import = pd.read_hdf(str_path_unc_res_all_annual, key = str_key_unc_res, 
                                     where = "(Flow = 'Import') & (Partner in list_iter_countries)").droplevel('Flow')
        print('Import dataset loaded')    
        ### Import data reverting:
        ser_unc_import.index.set_names('Partner_Inv', level = 1, inplace = True)
        ser_unc_import.index.set_names('Reporter', level = 2, inplace = True)
        ser_unc_import.index.set_names('Partner', level = 1, inplace = True)
        ser_unc_import = ser_unc_import.swaplevel('Reporter', 'Partner').sort_index()
        print('Import dataset reverted')
        ### Datasets concatenation:
        df_export_aug = pd.concat([ser_unc_export, ser_unc_import], axis = 1, names = 'Source Flow', keys = ['Export', 'Import']).astype('float32')
        del ser_unc_export
        del ser_unc_import    
        gc.collect()    
        print('Export and reverted Import dataset concatenated')
        ### Columns categorization:
        df_export_aug.to_hdf(str_path_unc_res_flows, key = str_key_unc_res, mode = 'a', format = 'table', complevel = 9, append = True)                
        print('Aggregated dataset added to container')
    #    break

In [23]:
### CIF COEFFICIENTS CALCULATION & IMPLEMENTATION

gc.collect()
### Files deleting:
if (os.path.exists(str_path_export_bilateral)):
    os.remove(str_path_export_bilateral)
if (os.path.exists(str_path_import_bilateral)):
    os.remove(str_path_import_bilateral)
### Getting list of commodities:
str_date = '2020-12-31'
list_commodity_id = sorted(pd.read_hdf(str_path_unc_res_flows, key = str_key_unc_res, where = "Date in str_date").index.get_level_values('Commodity_ID').unique())
### Bounds to filter bilateral Import to Export ratio before median calculation:
flo_lower_bound = 1.0
flo_upper_bound = 2.0
### Bilateral median calculation procedure:
def get_obs_median(df_comm):
    ### Export to Import ratio:
    ser_obs_coeff = df_comm['Import'] / df_comm['Export']
    ### Ratio filtering:
    ser_obs_coeff = ser_obs_coeff.loc[(ser_obs_coeff >= flo_lower_bound) & (ser_obs_coeff <= flo_upper_bound)]
    ### Filtered timeseries median as a result:
    return ser_obs_coeff.median()
### Calulation CIF coefficient for all commodities:
for iter_commodity in list_commodity_id:
    gc.collect()
    df_iter_flows = pd.read_hdf(str_path_unc_res_flows, key = str_key_unc_res, where = "Commodity_ID = iter_commodity")
    ser_cif_median = df_iter_flows.droplevel('Commodity_ID').groupby(['Reporter', 'Partner']).apply(get_obs_median)
    ### General commodity median calculation:
    flo_median = ser_cif_median.median()
    print(iter_commodity, ':', flo_median)
    ### Filling missed bilateral values with general commodity median:
    if not (np.isnan(flo_median)):
        ser_cif_median.fillna(flo_median, inplace = True)        
    ser_cif_median.name = 'CIF_Coefficient'              
    ### Adding CIF coefficients to dataset:
    df_export_cif = df_iter_flows.merge(ser_cif_median, left_index = True, right_index = True)
    df_export_cif = df_export_cif.reorder_levels(['Date', 'Reporter', 'Partner', 'Type', 'Commodity_ID'])
    ### Import correction:
    df_export_cif['Import_Corrected'] = df_export_cif['Import'] / df_export_cif['CIF_Coefficient']
    ### Export correction:
    df_export_cif['Export_Corrected'] = df_export_cif['Export'] * df_export_cif['CIF_Coefficient']
    ### Combining Export & Import data:
    ser_export_cif = df_export_cif['Export'].combine_first(df_export_cif['Import_Corrected']).astype('float32')
    ser_import_cif = df_export_cif['Import'].combine_first(df_export_cif['Export_Corrected']).astype('float32')
#    del df_export_cif
    gc.collect()
    ser_import_cif = ser_import_cif.reorder_levels(['Date', 'Partner', 'Reporter', 'Type', 'Commodity_ID']).sort_index()                               
    ser_import_cif.index.names = ['Date', 'Reporter', 'Partner', 'Type', 'Commodity_ID']
    gc.collect()
    ### Incorporation options:
    ser_export_cif.squeeze().to_hdf(str_path_export_bilateral, key = str_key_unc_export, mode = 'a', format = 'table', complevel = 9, append = True, 
                                    min_itemsize = {'Type': 8, 'Commodity_ID': 3})
    ser_import_cif.squeeze().to_hdf(str_path_import_bilateral, key = str_key_unc_import, mode = 'a', format = 'table', complevel = 9, append = True, 
                                    min_itemsize = {'Type': 8, 'Commodity_ID': 3}) 
#    break

206 : 1.3838366270065308
210 : 1.3869344592094421
214 : 1.399951994419098


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1112: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


218 : nan
219 : 1.3460965752601624
94 : 1.2919945120811462
95 : 1.3043478727340698
96 : 1.2878787517547607
97 : 1.3333333730697632
99 : 1.4108654856681824


In [27]:
### TEMP

ser_export_cif = pd.read_hdf(str_path_export_bilateral).sort_index()
ser_import_cif = pd.read_hdf(str_path_import_bilateral).sort_index()

Date        Reporter  Partner  Type   Commodity_ID
2008-12-31  AT        BE       Goods  95              11714.000000
                      ZM       Goods  95                  6.133333
            BE        AT       Goods  95               8966.000000
                      ZM       Goods  95                  3.000000
            ZM        AT       Goods  95                  0.000000
Name: Export, dtype: float32

In [42]:
### TEMP

display(ser_export_cif.loc['2008-12-31', ['AT', 'BE', 'AO'], ['AT', 'BE', 'AO'], 'Goods', '99'])
display(ser_import_cif.loc['2008-12-31', ['AT', 'BE', 'AO'], ['AT', 'BE', 'AO'], 'Goods', '99'] / 1.41)

Date        Reporter  Partner  Type   Commodity_ID
2008-12-31  AT        BE       Goods  99                 12.0
            BE        AO       Goods  99              26907.0
                      AT       Goods  99              89155.0
Name: Export, dtype: float32

Date        Reporter  Partner  Type   Commodity_ID
2008-12-31  AO        BE       Goods  99              26923.515625
            AT        BE       Goods  99                 80.851067
            BE        AT       Goods  99              20371.630859
Name: Import, dtype: float32

In [39]:
### TEMP

df_iter_flows.sort_index().loc[('2008-12-31', ['AT', 'BE', 'AO'], ['AT', 'BE', 'AO'], 'Goods', '99'), :]

Export   Import
Date       Reporter Partner Type  Commodity_ID                  
2008-12-31 AT       BE      Goods 99               12.0  28724.0
           BE       AO      Goods 99            26907.0      NaN
                    AT      Goods 99            89155.0    114.0